In [24]:
!pip install ipywidgets --upgrade

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/139.4 kB ? eta -:--:--
   ---------------------------------------- 0.0/139.4 kB ? eta -:--:--
   -- ------------------------------------- 10.2/139.4 kB ? eta -:--:--
   -------- ------------------------------ 30.7/139.4 kB 435.7 kB/s eta 0:00:01
   ----------------- --------------------- 61.4/139.4 kB 465.5 kB/s eta 0:00:01
   ---------------------- ---------------- 81.9/139.4 kB 416.7 kB/s eta 0:00:01
   ------------------------- ------------- 92.2/139.4 kB 403.5 kB/s eta 0:00:01
   ------------------------------------ - 133.1/139.4 kB 462.0 kB/s eta 0:00:01
   -------------------------------------- 139.4/139.4 kB 434.9 kB/s eta 0:00:00
   ---------------------------------------- 0.0/215.0 kB ? eta -:--:--
   ----- --------------------------------- 30.7/215.0 kB 660.6 kB/s eta 0:00:01
   ----------- --------------------------- 61.4/215.0 kB 656.4 kB/s eta 0:00:

In [35]:
pip install -U ipywidgets

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [1]:
#Подключение библиотек
import pandas as pd
import networkx as nx
from pyvis.network import Network
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import interact, widgets
import os

In [2]:
# Загрузка данных из датасета
data = pd.read_csv("imdb_movies.csv")

In [3]:
#Обработка датасета от пустых значений 

# Удаляем строки с пропущенными значениями
data_cleaned = data.dropna()

# Сбрасываем индексы
data_cleaned.reset_index(drop=True, inplace=True)

# Просматриваем первые 5 строк очищенных данных
data_cleaned.head(7)

,names,date_x,score,genre,overview,crew,orig_title,status,orig_lang,budget_x,revenue,country
0,Creed III,03/02/2023,73.0,"Drama, Action","After dominating the boxing world, Adonis Cree...","Michael B. Jordan, Adonis Creed, Tessa Thompso...",Creed III,Released,English,75000000.0,2.716167e+08,AU
1,Avatar: The Way of Water,12/15/2022,78.0,"Science Fiction, Adventure, Action",Set more than a decade after the events of the...,"Sam Worthington, Jake Sully, Zoe Saldaña, Neyt...",Avatar: The Way of Water,Released,English,460000000.0,2.316795e+09,AU
2,The Super Mario Bros. Movie,04/05/2023,76.0,"Animation, Adventure, Family, Fantasy, Comedy","While working underground to fix a water main,...","Chris Pratt, Mario (voice), Anya Taylor-Joy, P...",The Super Mario Bros. Movie,Released,English,100000000.0,7.244590e+08,AU
3,Mummies,01/05/2023,70.0,"Animation, Comedy, Family, Adventure, Fantasy","Through a series of unfortunate events, three ...","Óscar Barberán, Thut (voice), Ana Esther Albor...",Momias,Released,"Spanish, Castilian",12300000.0,3.420000e+07,AU
4,Supercell,03/17/2023,61.0,Action,Good-hearted teenager William always lived in ...,"Skeet Ulrich, Roy Cameron, Anne Heche, Dr Quin...",Supercell,Released,English,77000000.0,3.409420e+08,US
5,Cocaine Bear,02/23/2023,66.0,"Thriller, Comedy, Crime","Inspired by a true story, an oddball group of ...","Keri Russell, Sari, Alden Ehrenreich, Eddie, O...",Cocaine Bear,Released,English,35000000.0,8.000000e+07,AU
6,John Wick: Chapter 4,03/23/2023,80.0,"Action, Thriller, Crime","With the price on his head ever increasing, Jo...","Keanu Reeves, John Wick, Donnie Yen, Caine, Bi...",John Wick: Chapter 4,Released,English,100000000.0,3.513494e+08,AU


In [7]:
# Пользователь вводит имя актера
actor_name = input("Введите имя актера: ")

# Поиск фильмов, в которых участвует указанный актер
actor_movies = data[data['crew'].str.contains(actor_name, na=False)]

# Вывод найденных фильмов
if not actor_movies.empty:
    print(f"Фильмы, в которых участвует актер {actor_name}:")
    for idx, row in actor_movies.iterrows():
        print(f"Название: {row['names']}")
        print(f"Дата выпуска: {row['date_x']}")
        print(f"Жанр: {row['genre']}")
        print(f"Рейтинг: {row['score']}")
        print("-----")
else:
    print(f"Фильмы с участием актера {actor_name} не найдены.")

Введите имя актера:  Michael B. Jordan


Фильмы, в которых участвует актер Michael B. Jordan:
Название: Creed III
Дата выпуска: 03/02/2023 
Жанр: Drama, Action
Рейтинг: 73.0
-----
Название: Creed II
Дата выпуска: 12/06/2018 
Жанр: Drama
Рейтинг: 70.0
-----
Название: Creed
Дата выпуска: 11/26/2015 
Жанр: Drama
Рейтинг: 74.0
-----
Название: Black Panther
Дата выпуска: 02/15/2018 
Жанр: Action, Adventure, Science Fiction
Рейтинг: 74.0
-----
Название: Tom Clancy's Without Remorse
Дата выпуска: 04/30/2021 
Жанр: Action, Thriller
Рейтинг: 70.0
-----
Название: Just Mercy
Дата выпуска: 01/23/2020 
Жанр: Drama, Crime, History
Рейтинг: 79.0
-----
Название: Chronicle
Дата выпуска: 02/01/2012 
Жанр: Science Fiction, Drama, Thriller
Рейтинг: 68.0
-----
Название: Justice League: The Flashpoint Paradox
Дата выпуска: 07/30/2013 
Жанр: Science Fiction, Animation, Action, Adventure
Рейтинг: 79.0
-----
Название: Fantastic Four
Дата выпуска: 08/06/2015 
Жанр: Action, Adventure, Science Fiction
Рейтинг: 44.0
-----
Название: Fruitvale Station
Дата

In [12]:
# Функция для создания и отображения графа
def create_and_show_graph(actor_movies, min_rating=0):
    G = nx.Graph()
    
    for idx, row in actor_movies.iterrows():
        G.add_node(row['names'], date=row['date_x'], genre=row['genre'], score=row['score'])
    
    # Создание рёбер на основе общих актеров, исключая искомого актера
    for movie1 in G.nodes():
        for movie2 in G.nodes():
            if movie1 != movie2:
                actors_movie1 = set(data[data['names'] == movie1]['crew'].iloc[0].split(", "))
                actors_movie2 = set(data[data['names'] == movie2]['crew'].iloc[0].split(", "))
                common_actors = actors_movie1.intersection(actors_movie2)
                common_actors = [actor for actor in common_actors if actor != actor_name]  # Исключаем искомого актера
                if common_actors:
                    G.add_edge(movie1, movie2, common_actors=common_actors)
    
    # Создание интерактивного графа с помощью pyvis
    
    # Создание интерактивного графа с помощью pyvis
    nt = Network(height='800px', width='100%', notebook=True)
    nt.from_nx(G)
    
    # Добавление информации при наведении на ребро
    for edge in nt.edges:
        common_actors = ', '.join(edge['common_actors'])
        edge["title"] = f"Общие актеры: {common_actors}"
    
    # Добавление информации при наведении на узел
    for node in nt.nodes:
        node_info = f"<b>Дата выпуска:</b> {node['date']}<br><b>Жанр:</b> {node['genre']}<br><b>Рейтинг:</b> {node['score']}"
        node["title"] = node["label"] + "<br>" + node_info
    
    # Отображение графа
    nt.show('graph.html')
    print(f"Граф фильмов с участием актера {actor_name} и рейтингом не ниже {min_rating} создан. Откройте файл graph.html для просмотра.")
    # Сохранение графа в файл graph.html
    nt.save_graph("graph.html")

# Функция-обработчик нажатия кнопки обновления
def update_graph(b):
    create_and_show_graph(actor_movies, min_rating=min_rating_slider.value)


# Вызов функции создания и отображения графа для начального актера и минимального рейтинга
create_and_show_graph(actor_movies)

graph.html
Граф фильмов с участием актера Michael B. Jordan и рейтингом не ниже 0 создан. Откройте файл graph.html для просмотра.


In [13]:
# Создание виджетов для фильтрации по минимальному рейтингу и кнопки обновления
min_rating_slider = widgets.FloatSlider(value=0, min=0, max=100, step=0.1, description='Мин. рейтинг:')
update_button = widgets.Button(description="Обновить граф")

In [14]:
# Функция-обработчик нажатия кнопки обновления
def update_graph(b):
    create_and_show_graph(actor_movies, min_rating=min_rating_slider.value)

# Привязка функции-обработчика к кнопке
update_button.on_click(update_graph)

# Отображение виджетов
display(min_rating_slider)
display(update_button)

FloatSlider(value=0.0, description='Мин. рейтинг:')

Button(description='Обновить граф', style=ButtonStyle())